In [25]:
import json
import pandas as pd
import numpy as np

In [2]:
filename = "/Users/gandalf/Documents/data/data_report.json"
with open(filename) as fh:
    json_data = fh.read()
    data = json.loads(json_data)

In [3]:
df = pd.read_json(filename)

In [4]:
df.head(2)

,_acl,_auth_data_facebook,_created_at,_hashed_password,_id,_p_room,_rperm,_updated_at,_wperm,about,...,onboarded,petsOk,picture,positions,recommended,smokingOk,term,type,username,work
0,"{'07cSOmlL0F': {'w': True, 'r': True}, '*': {'...",NaN,{'$date': '2016-11-15T22:54:02.653Z'},$2a$10$Gx8s0PL/YvNfBwcEkPrG1.gGWIB5azU.WBpnc42...,07cSOmlL0F,NaN,"[*, 07cSOmlL0F]",{'$date': '2017-02-08T00:06:05.667Z'},[07cSOmlL0F],NaN,...,0.0,1.0,374b4b768793c7a03bc27e35f032bbe7_pic.jpeg,NaN,NaN,0.0,12.0,private,rlouw701@gmail.com,CSAA
1,"{'0PZ0jSiV11': {'w': True, 'r': True}, '*': {'...",NaN,{'$date': '2016-11-16T03:36:00.307Z'},$2a$10$vVgy1/lfyq06j9mQJDPxC.cNEZaF8b05eLJJGeX...,0PZ0jSiV11,Room$dANkaNpTN9,"[*, 0PZ0jSiV11]",{'$date': '2017-03-13T19:38:46.405Z'},[0PZ0jSiV11],NaN,...,0.0,1.0,3addf8af264017514233605b49675e53_pic.jpeg,NaN,NaN,1.0,NaN,NaN,rlouw777777@gmail.com,CSAA


In [ ]:
def get_user_from_json(df):
    '''
    original columns: 
    '_acl'                drop     UNKNOWN, ex: {'*': {'r': True}, '25mzfm6r5U': {'r': True, 'w': True}}
    '_auth_data_facebook' drop     FACEBOOK AUTHORIZATION, not helpful (maybe as a yes/no?)
    '_created_at'         to_date  DATE CREATED, format {'$date': '2016-11-15T22:54:02.653Z'}
    '_hashed_password'    PASSWORD, not useful
    '_id'                 string, IMPORTANT!
    '_p_room'             UNKNOWN, string, ex: 'Room$dANkaNpTN9' post a room, someone with a room available
    '_rperm'              UNKNOWN, ['*', '0PZ0jSiV11'] ignore
    '_updated_at'         DATE UPDATED, eg: {'$date': '2017-03-13T19:38:46.405Z'}
    '_wperm'              UNKNOWN, eg: ['0PZ0jSiV11'] ignore
    'about'               ABOUT ME SECTION, string
    'activeAt'            MOST RECENT ACTIVE format: {'$date': '2017-12-04T03:01:37.052Z'}
    'amenities'           WHICH AMENITIES, eg: [Laundry, A/C, Dishwasher, Private Bath]
    'available'           WHEN AVAILABLe, format: {'$date': '2017-01-31T11:40:52.000Z'}
    'birthday'            BIRTHDAY, eg: {'$date': '1992-01-01T00:00:00.000Z'}
    'blocked'             EVER BLOCKED eg: [{'__type': 'Pointer', 'className': '_User', 'objectId': 'nHcaVTlJUg'}]
    'college'             COLLEGE, string
    'email'               EMAIL, string
    'emailVerified'       ???, just NaN and 0 ignore
    'facebookId'          18-long num, or NaN
    'firstName'           FIRST NAME (string)
    'foundRoommate'       NaN or 1 (ignore)
    'gender'              GENDER, male, female, nan
    'groupChat'           ??? NaN or 1
    'hobbies'             HOBBIES eg: [Foodie, Movies, Socializing]
    'hometown'            HOMETOWN, mostly NAN, has city, state, country
                          'hometownCity', 'hometownCountry','hometownCounty', 'hometownState'
    'inRelationship', 
    'isClean',
    'isNight', 
    'isStudent', 
    'lastName', 
    'likes'               CHECK BACK facebook likes,eg a band or place
    'linkedinId'
    'location'            LATITUDE AND LONGITUDE
    'maxCost', 'minCost', 
    'neighborhoods'       eg: [{'__type': 'Pointer', 'className': 'Neighborhood', 'objectId': 'rQgVwGMItl'}]
    'numRoommates' 
    'onboarded'           SUPA IMPORT!!!
    'petsOk', 
    'picture'             string, eg: '6c5316ca545062ad708899e89adb917a_pic.jpeg'
    'positions'           some sort of list, picture positions, for posting a room
    'recommended'         ignore, internal use, for caching users
    'smokingOk'
    'term',
    'type',               house type ( 
    'username', 
    'work']
    
    '''
    
    df._created_at = df._created_at.apply(lambda x: pd.to_datetime(x['$date']))
    df._p_room = df._p_room.apply(lambda x: isinstance(x,str))
    df._updated_at = df._updated_at.apply(lambda x: pd.to_datetime(x['$date']))
    df.activeAt = df.activeAt.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.available = df.available.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.birthday = df.birthday.apply(lambda x: pd.to_datetime(x['$date']) if isinstance(x,str) else 0)
    df.email = df.email.apply(lambda x: True if isinstance(x,str) else False)
    df.facebookId = df.facebookId.apply(lambda x: True if isinstance(x,int) else False)
    df.likes = df.likes.apply(lambda x: True if isinstance(x,int) else False)
    df.linkedinId = df.linkedinId.apply(lambda x: True if isinstance(x,int) else False)
    df.picture = df.picture.apply(lambda x: True if isinstance(x,str) else False)
    df['work_tf'] = df.work.apply(lambda x: True if isinstance(x,str) else False)
    col_to_drop = ['_acl','_auth_data_facebook','_hashed_password','_rperm','_wperm','blocked','emailVerified','firstName',
                  'foundRoommate','groupChat','hometown','hometownCounty','lastName','positions','recommended','username']
    df = df.fillna({'about':''})

    df = df.drop(col_to_drop, axis = 1)
    
    return df

temp_df = df.copy()
temp_df = get_user_from_json(temp_df)

In [ ]:
temp_df.iloc[:,39:].head()

In [34]:
type(temp_df._created_at[0])

pandas._libs.tslib.Timestamp

firebase - conversations
mongodb for others